In [11]:
%pip install conllu torch scikit-learn -q


In [12]:
from google.colab import drive
drive.mount('/content/drive')


Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [13]:
import os
print(os.listdir("/content/drive/MyDrive/NLP/NLM_2/data_set"))


['ar_padt-ud-dev.conllu', 'ar_padt-ud-test.conllu', 'ar_padt-ud-train.conllu']


In [14]:
import torch
import torch.nn as nn
from torch.utils.data import Dataset, DataLoader
from collections import Counter
from conllu import parse_incr
from sklearn.metrics import classification_report
import random
import numpy as np

# تثبيت العشوائية علشان النتائج قابلة لإعادة الإنتاج
SEED = 42
random.seed(SEED)
np.random.seed(SEED)
torch.manual_seed(SEED)
torch.cuda.manual_seed_all(SEED)

def load_ud_conllu(filepath):
    with open(filepath, "r", encoding="utf-8") as f:
        data = list(parse_incr(f))
    sents = []
    for sent in data:
        toks = []
        for tok in sent:
            upos = tok.get("upos")
            if upos:
                toks.append((tok["form"], upos))
        if toks:
            sents.append(toks)
    return sents


In [15]:
train_sents = load_ud_conllu("/content/drive/MyDrive/NLP/NLM_2/data_set/ar_padt-ud-train.conllu")
dev_sents   = load_ud_conllu("/content/drive/MyDrive/NLP/NLM_2/data_set/ar_padt-ud-dev.conllu")
test_sents  = load_ud_conllu("/content/drive/MyDrive/NLP/NLM_2/data_set/ar_padt-ud-test.conllu")

print("Train / Dev / Test sentences:",
      len(train_sents), len(dev_sents), len(test_sents))
print("Example sentence:", train_sents[0][:10])


Train / Dev / Test sentences: 6075 909 680
Example sentence: [('برلين', 'X'), ('ترفض', 'VERB'), ('حصول', 'NOUN'), ('شركة', 'NOUN'), ('اميركية', 'ADJ'), ('على', 'ADP'), ('رخصة', 'NOUN'), ('تصنيع', 'NOUN'), ('دبابة', 'NOUN'), ('"', 'PUNCT')]


In [16]:
# نبني vocab للكلمات والـ POS tags
word_counter = Counter()
tag_counter = Counter()

for sents in [train_sents, dev_sents, test_sents]:
    for sent in sents:
        for w, t in sent:
            word_counter[w] += 1
            tag_counter[t] += 1

PAD = "<PAD>"
UNK = "<UNK>"

# word2id
word2id = {PAD: 0, UNK: 1}
for w, freq in word_counter.items():
    if freq >= 2:          # نرمي الكلمات النادرة جداً (تقدرين تغيّرينها)
        word2id[w] = len(word2id)

# tag2id
tag2id = {tag: i for i, tag in enumerate(sorted(tag_counter.keys()))}
id2tag = {i: t for t, i in tag2id.items()}

print("Vocab size:", len(word2id))
print("Tags:", tag2id)

def encode_sentence(sent, word2id, tag2id):
    word_ids = []
    tag_ids = []
    for w, t in sent:
        word_ids.append(word2id.get(w, word2id[UNK]))
        tag_ids.append(tag2id[t])
    return word_ids, tag_ids

encoded_train = [encode_sentence(s, word2id, tag2id) for s in train_sents]
encoded_dev   = [encode_sentence(s, word2id, tag2id) for s in dev_sents]
encoded_test  = [encode_sentence(s, word2id, tag2id) for s in test_sents]

print("Encoded train example:", encoded_train[0][0][:10], "...")


Vocab size: 18420
Tags: {'ADJ': 0, 'ADP': 1, 'ADV': 2, 'AUX': 3, 'CCONJ': 4, 'DET': 5, 'INTJ': 6, 'NOUN': 7, 'NUM': 8, 'PART': 9, 'PRON': 10, 'PROPN': 11, 'PUNCT': 12, 'SCONJ': 13, 'SYM': 14, 'VERB': 15, 'X': 16, '_': 17}
Encoded train example: [2, 3, 4, 5, 6, 7, 8, 9, 10, 11] ...


In [17]:
class POSDataset(Dataset):
    def __init__(self, encoded_sents):
        self.data = encoded_sents

    def __len__(self):
        return len(self.data)

    def __getitem__(self, idx):
        word_ids, tag_ids = self.data[idx]
        return torch.tensor(word_ids, dtype=torch.long), torch.tensor(tag_ids, dtype=torch.long)

def collate_fn(batch):
    word_seqs, tag_seqs = zip(*batch)
    lengths = [len(s) for s in word_seqs]
    max_len = max(lengths)

    padded_words = []
    padded_tags = []

    for w_seq, t_seq in zip(word_seqs, tag_seqs):
        pad_len = max_len - len(w_seq)

        padded_words.append(
            torch.cat([w_seq, torch.full((pad_len,), word2id[PAD], dtype=torch.long)])
        )
        padded_tags.append(
            torch.cat([t_seq, torch.full((pad_len,), -100, dtype=torch.long)])  # -100 = ignore_index في loss
        )

    return torch.stack(padded_words), torch.stack(padded_tags), torch.tensor(lengths)

train_dataset = POSDataset(encoded_train)
dev_dataset   = POSDataset(encoded_dev)
test_dataset  = POSDataset(encoded_test)

train_loader = DataLoader(train_dataset, batch_size=32, shuffle=True,  collate_fn=collate_fn)
dev_loader   = DataLoader(dev_dataset,   batch_size=32, shuffle=False, collate_fn=collate_fn)
test_loader  = DataLoader(test_dataset,  batch_size=32, shuffle=False, collate_fn=collate_fn)

print("Batches -> train/dev/test:",
      len(train_loader), len(dev_loader), len(test_loader))


Batches -> train/dev/test: 190 29 22


In [18]:
class BiLSTMPOS(nn.Module):
    def __init__(self, vocab_size, embed_dim, hidden_dim, num_tags, pad_idx):
        super().__init__()
        self.embedding = nn.Embedding(vocab_size, embed_dim, padding_idx=pad_idx)
        self.lstm = nn.LSTM(embed_dim, hidden_dim, num_layers=1,
                            bidirectional=True, batch_first=True)
        self.fc = nn.Linear(hidden_dim * 2, num_tags)

    def forward(self, input_ids):
        x = self.embedding(input_ids)         # [B, L, E]
        outputs, _ = self.lstm(x)             # [B, L, 2H]
        logits = self.fc(outputs)             # [B, L, num_tags]
        return logits

vocab_size = len(word2id)
num_tags = len(tag2id)
pad_idx = word2id[PAD]

device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
model = BiLSTMPOS(vocab_size, embed_dim=128, hidden_dim=256,
                  num_tags=num_tags, pad_idx=pad_idx).to(device)

print("Using device:", device)


Using device: cpu


In [19]:
import torch.optim as optim

criterion = nn.CrossEntropyLoss(ignore_index=-100)
optimizer = optim.Adam(model.parameters(), lr=1e-3)

def train_one_epoch(model, loader):
    model.train()
    total_loss = 0.0
    for words, tags, lengths in loader:
        words = words.to(device)
        tags = tags.to(device)

        optimizer.zero_grad()
        logits = model(words)  # [B, L, num_tags]

        loss = criterion(
            logits.view(-1, num_tags),
            tags.view(-1)
        )
        loss.backward()
        optimizer.step()

        total_loss += loss.item()
    return total_loss / len(loader)

def eval_accuracy(model, loader):
    model.eval()
    correct = 0
    total = 0
    with torch.no_grad():
        for words, tags, lengths in loader:
            words = words.to(device)
            tags = tags.to(device)
            logits = model(words)
            preds = logits.argmax(dim=-1)

            mask = tags != -100
            correct += (preds[mask] == tags[mask]).sum().item()
            total += mask.sum().item()
    return correct / total if total > 0 else 0.0

EPOCHS = 5

for epoch in range(1, EPOCHS + 1):
    train_loss = train_one_epoch(model, train_loader)
    dev_acc = eval_accuracy(model, dev_loader)
    print(f"Epoch {epoch:02d} | train loss: {train_loss:.4f} | dev acc: {dev_acc:.4f}")

test_acc = eval_accuracy(model, test_loader)
print("\nFINAL test accuracy:", test_acc)


Epoch 01 | train loss: 0.8980 | dev acc: 0.8359
Epoch 02 | train loss: 0.3753 | dev acc: 0.8836
Epoch 03 | train loss: 0.2503 | dev acc: 0.9067
Epoch 04 | train loss: 0.1761 | dev acc: 0.9186
Epoch 05 | train loss: 0.1270 | dev acc: 0.9233

FINAL test accuracy: 0.9264884379571131


In [21]:
def get_predictions_and_labels(model, loader):
    model.eval()
    all_preds = []
    all_labels = []

    with torch.no_grad():
        for words, tags, lengths in loader:
            words = words.to(device)
            tags = tags.to(device)

            logits = model(words)
            preds = logits.argmax(dim=-1)

            mask = tags != -100
            all_preds.extend(preds[mask].cpu().numpy())
            all_labels.extend(tags[mask].cpu().numpy())

    return all_labels, all_preds


# احصل على النتائج
y_true, y_pred = get_predictions_and_labels(model, test_loader)

# إصلاح الخطأ هنا 👇
unique_labels = sorted(set(y_true + y_pred))
target_names = [id2tag[i] for i in unique_labels]

from sklearn.metrics import classification_report

report = classification_report(
    y_true,
    y_pred,
    labels=unique_labels,
    target_names=target_names,
    digits=3
)

print(report)


              precision    recall  f1-score   support

         ADJ      0.851     0.843     0.847      2937
         ADP      0.987     0.990     0.988      4528
         ADV      0.940     0.904     0.922       104
         AUX      0.899     0.766     0.827       197
       CCONJ      0.992     0.984     0.988      1962
         DET      0.929     0.952     0.941       623
        NOUN      0.918     0.927     0.922      9547
         NUM      0.923     0.873     0.897       779
        PART      0.914     0.889     0.901       226
        PRON      0.992     0.997     0.995      1133
       PROPN      1.000     0.290     0.450        31
       PUNCT      1.000     0.999     0.999      2052
       SCONJ      0.976     0.993     0.984       535
         SYM      1.000     0.951     0.975        41
        VERB      0.836     0.853     0.845      2191
           X      0.624     0.596     0.610      1380
           _      0.993     0.999     0.996      3865

    accuracy              

In [22]:
def tag_sentence(sentence, model, word2id, id2tag):
    model.eval()
    words = sentence.split()
    word_ids = [word2id.get(w, word2id[UNK]) for w in words]
    inp = torch.tensor(word_ids, dtype=torch.long).unsqueeze(0).to(device)

    with torch.no_grad():
        logits = model(inp)
        preds = logits.argmax(dim=-1).squeeze(0).cpu().tolist()

    tags = [id2tag[p] for p in preds]
    return list(zip(words, tags))

example_sentence = "كان الجو جميلاً في المدينة القديمة"
tag_sentence(example_sentence, model, word2id, id2tag)


[('كان', 'VERB'),
 ('الجو', 'NOUN'),
 ('جميلاً', 'ADJ'),
 ('في', 'ADP'),
 ('المدينة', 'NOUN'),
 ('القديمة', 'ADJ')]

In [23]:
def tag_sentence(sentence, model, word2id, id2tag):
    model.eval()
    words = sentence.split()
    word_ids = [word2id.get(w, word2id["<UNK>"]) for w in words]

    inp = torch.tensor(word_ids, dtype=torch.long).unsqueeze(0).to(device)

    with torch.no_grad():
        logits = model(inp)
        preds = logits.argmax(dim=-1).squeeze(0).cpu().tolist()

    tags = [id2tag[p] for p in preds]
    return words, tags


In [24]:
def print_tagged_sentences(sentences, model, word2id, id2tag):
    for sent in sentences:
        words, tags = tag_sentence(sent, model, word2id, id2tag)
        print("Sentence:")
        for w, t in zip(words, tags):
            print(f"{w:<15} {t}")
        print("*" * 50)
demo_sentences = [
    "كان الجو جميلا في المدينة القديمة",
    "سافر أحمد إلى القاهرة لمقابلة صديقه",
    "بدأ العرض في السينما مساء الجمعة"
]

print_tagged_sentences(demo_sentences, model, word2id, id2tag)


Sentence:
كان             VERB
الجو            NOUN
جميلا           ADJ
في              ADP
المدينة         NOUN
القديمة         ADJ
**************************************************
Sentence:
سافر            VERB
أحمد            X
إلى             ADP
القاهرة         NOUN
لمقابلة         X
صديقه           _
**************************************************
Sentence:
بدأ             VERB
العرض           NOUN
في              ADP
السينما         NOUN
مساء            NOUN
الجمعة          NOUN
**************************************************
